In [1]:
import rasterio
import joblib
import numpy as np
import pandas as pd
import os
import zipfile
from scipy.ndimage import zoom

# --- CONFIGURATION ---
model_path = "../models/manzanillo_biomass_model.joblib"
inegi_path = "../data/raw/inegi_cem/Colima_r15m.tif"
sentinel_dir = "../data/raw/sentinel/"
sentinel_zip = [f for f in os.listdir(sentinel_dir) if f.endswith('.zip')][0]
zip_path = os.path.join(sentinel_dir, sentinel_zip)
output_targets = "../data/processed/quantum_targets.csv"

print("🚀 PILOT 2: TARGET EXTRACTION")

# 1. RE-GENERATE RISK MAP (Fast Mode)
# We repeat the fast mapping logic to get the data array
print("1. Re-calculating Risk Surface...")
model = joblib.load(model_path)
sample_rate = 5 

with rasterio.open(inegi_path) as src:
    dem = src.read(1)[::sample_rate, ::sample_rate]
    transform = src.transform
    # Adjust transform for sampling (We need this to get real Lat/Lons later)
    new_transform = transform * transform.scale(sample_rate, sample_rate)
    
    px, py = src.res
    px, py = px * sample_rate, py * sample_rate
    dy, dx = np.gradient(dem, -py, px)
    slope = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))

with zipfile.ZipFile(zip_path, 'r') as z:
    red_file = [f for f in z.namelist() if "B04" in f and "10m" in f][0]
    nir_file = [f for f in z.namelist() if "B08" in f and "10m" in f][0]
    with rasterio.open(f"/vsizip/{zip_path}/{red_file}") as r: red = r.read(1)
    with rasterio.open(f"/vsizip/{zip_path}/{nir_file}") as n: nir = n.read(1)

zoom_y = dem.shape[0] / red.shape[0]
zoom_x = dem.shape[1] / red.shape[1]
red_small = zoom(red, (zoom_y, zoom_x), order=0)
nir_small = zoom(nir, (zoom_y, zoom_x), order=0)

num = (nir_small - red_small).astype(float)
den = (nir_small + red_small).astype(float)
ndvi = np.divide(num, den, out=np.zeros_like(num), where=den!=0)

input_df = pd.DataFrame({
    'Elevation_m': dem.flatten(),
    'Slope_deg': slope.flatten(),
    'NDVI': ndvi.flatten()
}).fillna(0)

pred_biomass = model.predict(input_df)
biomass_map = pred_biomass.reshape(dem.shape)
risk_map = (biomass_map * slope) / (ndvi + 0.2)
risk_map = np.where(risk_map < 0, 0, risk_map)
risk_map = (risk_map / np.max(risk_map)) * 100

# 2. EXTRACT HIGH VALUE TARGETS
print("\n2. Identifying Top 50 Critical Targets...")

# Flatten the map again but keep indices
flat_risk = risk_map.flatten()
# Get indices of the top 50 riskiest pixels
# argsort sorts low-to-high, so we take the last 50 and reverse them
top_indices = np.argsort(flat_risk)[-50:][::-1]

targets = []
for idx in top_indices:
    # Convert flat index back to Row/Col
    row = idx // risk_map.shape[1]
    col = idx % risk_map.shape[1]
    
    # Convert Row/Col to Lat/Lon using the Transform
    # Rasterio transform: (col, row) -> (x, y)
    lon, lat = rasterio.transform.xy(new_transform, row, col, offset='center')
    
    score = flat_risk[idx]
    
    targets.append({
        'Target_ID': len(targets),
        'Latitude': lat,
        'Longitude': lon,
        'Risk_Score': score
    })

df_targets = pd.DataFrame(targets)
df_targets.to_csv(output_targets, index=False)

print("-" * 30)
print(f"✅ TARGET LIST GENERATED.")
print(f"   Saved {len(df_targets)} targets to: {output_targets}")
print(f"   Highest Risk Score: {df_targets['Risk_Score'].max():.1f}")
print("-" * 30)
print(df_targets.head())

🚀 PILOT 2: TARGET EXTRACTION
1. Re-calculating Risk Surface...

2. Identifying Top 50 Critical Targets...
------------------------------
✅ TARGET LIST GENERATED.
   Saved 50 targets to: ../data/processed/quantum_targets.csv
   Highest Risk Score: 100.0
------------------------------
   Target_ID   Latitude   Longitude  Risk_Score
0          0  19.079503 -103.656211  100.000000
1          1  19.077420 -103.653433   99.976265
2          2  19.079503 -103.652738   99.864363
3          3  19.078809 -103.658988   99.771706
4          4  19.074642 -103.654822   99.757631
